# Advent of Code 2022

## Day 16

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

### Imports

In [ ]:
from collections import namedtuple, defaultdict, deque
import re
from tqdm.notebook import tqdm as tqdm
from typing import Iterator, Optional, Union, Any
from collections import namedtuple
import re
import functools
from time import sleep
from math import inf

In [ ]:
def read_input(filename: str) -> Iterator[tuple[str, int, list[str]]]:
    with open(filename) as file:
        for line in file:
            line = line.strip()
            match = re.search(r'^Valve ([A-Z][A-Z]) has flow rate=([0-9][0-9]?); tunnel[a-z]* lead[a-z]* to valve[a-z]* ([A-Z, ]*)$', line)
            if match is None:
                raise ValueError(f're.search returned {match} on string "{line}"')
            if len(match.groups()) != 3:
                raise ValueError(f're.search returned {len(match.groups())} match grounps on string "{line}"')
            valve_number, flow_rate, leads_to = match.groups()
            yield valve_number, int(flow_rate), leads_to.split(', ')

In [ ]:
INPUT_FILE = 'data/test16.txt'
START = 'AA'

In [ ]:
key_nodes = {}
neighbours_raw = {}
for valve_number, flow_rate, leads_to in read_input(INPUT_FILE):
    if valve_number == START or flow_rate != 0:
        key_nodes[valve_number] = flow_rate
    neighbours_raw[valve_number] = leads_to

In [ ]:
def search(path, shortest):

    if path:

        # process path
        length = len(path) - 1
        head, tail = path[0], path[-1]
        assert (type(head) == str), f'{head = }, {path = }'
        assert (type(tail) == str), f'{tail = }, {path = }'
        best = shortest[(head, tail)]
        if length < best:
            shortest[(head, tail)] = length

            # all next neighbours
            for neighbour in neighbours_raw[tail]:
                search(path + [neighbour], shortest)

    else:

        # all start points
        for neighbour in neighbours_raw.keys():
            search([neighbour], shortest)

shortest = defaultdict(lambda: inf)
search([], shortest)

distance_matrix = {}
for x in sorted(key_nodes.keys()):
    distance_matrix[x] = {}
    for y in sorted(key_nodes.keys()):
        distance_matrix[x][y] = shortest[(x, y)]

In [ ]:
print('Nodes we can visit, with amount to vent:')
print(key_nodes)
print()

print('Shortest path distance between all pairs of nodes:')
for x in distance_matrix:
    print(x, distance_matrix[x])
print()

In [ ]:
# for a given sequence of vents, returns the total score
def score_sequence(sequence: list[str], total_time: int) -> Optional[int]:
    assert len(set(sequence)) == len(sequence)
    current_vertex = 'AA'
    time_remaining = total_time
    vented = 0
    for next_vertex in sequence:
        distance = distance_matrix[current_vertex][next_vertex]
        if time_remaining >= distance:
            current_vertex = next_vertex
            time_remaining -= distance
            if time_remaining == 1:
                return None
            elif time_remaining >= 0:
                time_remaining -= 1
                vented += key_nodes[current_vertex] * time_remaining
            else:
                return None
        else:
            return None
    return vented

In [ ]:
if 'test' in INPUT_FILE:
    example_sequence = ['DD', 'BB', 'JJ', 'HH', 'EE', 'CC']
    print(score_sequence([], 30))
    print(score_sequence(['DD'], 30))
    print(score_sequence(['DD', 'BB'], 30))
    print(score_sequence(['DD', 'BB', 'JJ'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH', 'EE'], 30))
    print(score_sequence(['DD', 'BB', 'JJ', 'HH', 'EE', 'CC'], 30))
else:
    print('running on real file')

In [ ]:
BEST_SCORE = 0
BEST_SEQUENCE = []

def track_best(score, sequence):
    global BEST_SCORE
    global BEST_SEQUENCE
    if score > BEST_SCORE:
        BEST_SCORE = score
        BEST_SEQUENCE = sequence
        print(BEST_SCORE, BEST_SEQUENCE)


def search_problem(sequence: list[str], total_time: int, closed: set[str], callback):

    score = score_sequence(sequence, total_time=total_time)

    if score is None:
        return

    callback(score, sequence)

    for next_vent in closed:
        search_problem(sequence=sequence+[next_vent], total_time=total_time, closed=closed-{next_vent}, callback=callback)


search_problem(sequence=[], total_time=30, closed=set(key_nodes.keys())-{'AA'}, callback=track_best)
print()
print('DONE')
print()
print(f'Best Score : {BEST_SCORE}')
print(f'Best Sequence : {BEST_SEQUENCE}')

## Part 2